# 作業 : Kaggle期中考 考ML與調參相關

In [28]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


data_path = 'data/3rd-ml100marathon-midterm/'
df_train = pd.read_csv(data_path + 'train_data.csv')
df_test = pd.read_csv(data_path + 'test_features.csv')
train_Y = df_train['poi']
train_num = train_Y.shape[0]

name = df_test['name']

train = df_train.drop(['name','poi'], axis=1)
test = df_test.drop(['name'],axis=1)
df = pd.concat([train, test])

In [3]:
df.shape

(146, 20)

In [4]:
df_test.head()

,name,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,...,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,BELDEN TIMOTHY N,5249999.0,2144013.0,-2334434.0,NaN,tim.belden@enron.com,953136.0,17355.0,484.0,228.0,...,NaN,NaN,210698.0,157569.0,NaN,213999.0,5521.0,7991.0,5501630.0,1110705.0
1,BOWEN JR RAYMOND M,1350000.0,NaN,-833.0,NaN,raymond.bowen@enron.com,NaN,65907.0,27.0,140.0,...,NaN,974293.0,1621.0,252055.0,NaN,278601.0,1593.0,1858.0,2669589.0,252055.0
2,HANNON KEVIN P,1500000.0,NaN,-3117011.0,NaN,kevin.hannon@enron.com,5538001.0,34039.0,32.0,32.0,...,NaN,1617011.0,11350.0,853064.0,NaN,243293.0,1035.0,1045.0,288682.0,6391065.0
3,DELAINEY DAVID W,3000000.0,NaN,NaN,NaN,david.delainey@enron.com,2291113.0,86174.0,3069.0,66.0,...,NaN,1294981.0,1661.0,1323148.0,NaN,365163.0,2097.0,3093.0,4747979.0,3614261.0
4,CAUSEY RICHARD A,1000000.0,NaN,-235000.0,NaN,richard.causey@enron.com,NaN,30674.0,49.0,58.0,...,NaN,350000.0,307895.0,2502063.0,NaN,415189.0,1585.0,1892.0,1868758.0,2502063.0


In [5]:
print("Size of train data:", df_train.shape)
print("Size of test data:", df_test.shape)
print("Size of df data:", df.shape)

Size of train data: (113, 22)
Size of test data: (33, 21)
Size of df data: (146, 20)


# EDA

In [6]:
print(df.dtypes.value_counts())
print(df.dtypes.value_counts)

float64    19
object      1
dtype: int64
<bound method IndexOpsMixin.value_counts of bonus                        float64
deferral_payments            float64
deferred_income              float64
director_fees                float64
email_address                 object
exercised_stock_options      float64
expenses                     float64
from_messages                float64
from_poi_to_this_person      float64
from_this_person_to_poi      float64
loan_advances                float64
long_term_incentive          float64
other                        float64
restricted_stock             float64
restricted_stock_deferred    float64
salary                       float64
shared_receipt_with_poi      float64
to_messages                  float64
total_payments               float64
total_stock_value            float64
dtype: object>


In [7]:
df.isnull().sum().sort_values(ascending=False)

loan_advances                142
director_fees                129
restricted_stock_deferred    128
deferral_payments            107
deferred_income               97
long_term_incentive           80
bonus                         64
from_messages                 60
from_poi_to_this_person       60
from_this_person_to_poi       60
shared_receipt_with_poi       60
to_messages                   60
other                         53
expenses                      51
salary                        51
exercised_stock_options       44
restricted_stock              36
email_address                 35
total_payments                21
total_stock_value             20
dtype: int64

In [8]:
df.select_dtypes(include=["float64"]).apply(pd.Series.nunique, axis = 0)

bonus                         41
deferral_payments             39
deferred_income               44
director_fees                 17
exercised_stock_options      101
expenses                      94
from_messages                 64
from_poi_to_this_person       57
from_this_person_to_poi       41
loan_advances                  4
long_term_incentive           52
other                         92
restricted_stock              97
restricted_stock_deferred     18
salary                        94
shared_receipt_with_poi       83
to_messages                   86
total_payments               125
total_stock_value            124
dtype: int64

# 測試  df_float

In [11]:
#df_float 空值補平均值
df_float = df.select_dtypes(include=["float64"])
df_float_mean = df_float.fillna(df_float.mean())

train_X = df_float_mean[:train_num]
lr = LogisticRegression()
cross_val_score(lr, train_X, train_Y, cv=5).mean()

0.8754940711462451

In [30]:
#df_float 空值補平均值 搭配最大最小化
MMEncoder = MinMaxScaler()
df_float_mms = MMEncoder.fit_transform(df_float_mean)

train_X = df_float_mms[:train_num]
test_X = df_float_ss[train_num:]

In [31]:
#df_float 空值補平均值 搭配標準化
#SSEncoder = StandardScaler()
#df_float_ss = SSEncoder.fit_transform(df_float_mean)

#train_X = df_float_ss[:train_num]
#lr = LogisticRegression()
#cross_val_score(lr, train_X, train_Y, cv=5).mean()

In [29]:
#線性回歸
lr = LogisticRegression()
cross_val_score(lr, train_X, train_Y, cv=5).mean()

0.8853754940711462

In [34]:
lr.fit(train_X, train_Y)
lr_pred = lr.predict_proba(test_X)[:,1]
sub = pd.DataFrame({'name':name, 'poi':lr_pred})
sub.to_csv('midterm-lr-1.csv', index=False)

In [32]:
#梯度提升機
gdbt = GradientBoostingClassifier()
cross_val_score(gdbt, train_X, train_Y, cv=5).mean()

0.8225296442687748

In [35]:
gdbt.fit(train_X, train_Y)
gdbt_pred = gdbt.predict_proba(test_X)[:,1]
sub = pd.DataFrame({'name':name, 'poi':gdbt_pred})
sub.to_csv('midterm-gdbt.csv', index=False)

In [33]:
#隨機森林
rf = RandomForestClassifier()
cross_val_score(rf, train_X, train_Y, cv=5).mean()

0.867588932806324

In [36]:
rf.fit(train_X, train_Y)
rf_pred = rf.predict_proba(test_X)[:,1]
sub = pd.DataFrame({'name':name, 'poi':rf_pred})
sub.to_csv('midterm-rf.csv', index=False)